In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from merge.preprocessing.tokenizer import AutoTokenizer


tokenizer = AutoTokenizer.create("bert-base-uncased")

In [ ]:
tokenizer = AutoTokenizer.create("bert-base-uncased")

In [ ]:
tokenizer.encode("Hello, my dog is cute")

In [ ]:
fw = load_dataset("roneneldan/TinyStories", split="train", cache_dir="./hf_cache", num_proc=16)


In [1]:
from merge.preprocessing.tokenizer import AutoTokenizer

tokenizer = AutoTokenizer.create("EleutherAI/gpt-neo-125m")
tokenizer.add_special_tokens({"pad_token": "<pad>"})

/home/leon/git/leon/merge/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1

In [1]:
from merge.preprocessing.tokenizer import AutoTokenizer

tokenizer = AutoTokenizer.create("microsoft/phi-2")


/Users/leonericsson/projects/github/merge/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")

tokenizer.bos_token_id

50256

In [7]:
from merge.modules.attention import causal_attention_mask

mask = causal_attention_mask(5)

In [ ]:
import torch
attention_mask = torch.tensor([1, 1, 1, 0, 0]).repeat(5, 1)

attention_mask = attention_mask.repeat(5, 1)

In [45]:
from merge.preprocessing.tokenizer import AutoTokenizer
tokenizer = AutoTokenizer.create("EleutherAI/gpt-neo-125m")

/Users/leonericsson/projects/github/merge/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [48]:
tokenizer.add_special_tokens({"pad_token": "<|pad|>"})


1

In [50]:
tokenizer.pad_token_id

50257

In [40]:
import torch
import torch.nn.functional as F

# Let's simulate a vocabulary where:
# pad_token_id = 0
# a_token_id = 1
# b_token_id = 2
# vocab_size = 3

# Original sequence: "aab" padded to length 5
original = torch.tensor([[1, 1, 2, 0, 0, 0]])  # [a, a, b, pad, pad]
input_ids = original[:, :-1]  # [a, a, b, pad, pad]
target_ids = original[:, 1:] # [a, b, pad, pad, pad]



In [41]:
input_ids, target_ids

(tensor([[1, 1, 2, 0, 0]]), tensor([[1, 2, 0, 0, 0]]))

In [42]:
# Create fake logits - we'll make them "perfect" predictions
# Shape: (batch=1, seq=5, vocab=3)
logits = torch.zeros(1, 5, 3)
# For each position, make the correct prediction have highest probability
logits[0, 0, 1] = 10.0  # predict 'a' at position 0
logits[0, 1, 2] = 10.0  # predict 'b' at position 1
# We don't care about the predictions for padded positions

# Test both with and without ignore_index
criterion_with_ignore = torch.nn.CrossEntropyLoss(ignore_index=0)
criterion_without_ignore = torch.nn.CrossEntropyLoss()

In [43]:
# Reshape for loss calculation
batch, seq, vocab = logits.shape
logits_reshaped = logits.reshape(batch * seq, vocab)
target_ids_reshaped = target_ids.reshape(batch * seq)

loss_with_ignore = criterion_with_ignore(logits_reshaped, target_ids_reshaped)
loss_without_ignore = criterion_without_ignore(logits_reshaped, target_ids_reshaped)

print(f"Loss with ignore_index=0: {loss_with_ignore.item()}")
print(f"Loss without ignore_index: {loss_without_ignore.item()}")

Loss with ignore_index=0: 9.083335316972807e-05
Loss without ignore_index: 0.6592037081718445
